In [1]:
pred_values = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [2]:
import os
import numpy as np
from scipy.io import wavfile
import pandas as pd

import operator
from pydub import AudioSegment
from audio.processor import WavProcessor, format_predictions
from collections import Counter

In [4]:
df = pd.read_csv('labels.csv')
df = df.set_index('index')

sig = AudioSegment.from_file('farm1.wav', format="wav")

window_size = 6 * 1000
overlap = 3 * 1000
fragment_length = 900 * 1000

current_length = 0

window_start = 0
window_end = window_start + window_size

codes = []

def get_labels(idx): 
    return [0.0 if val != 1 else 1.0 for val in df.loc[idx].values]

while current_length < fragment_length:
    print(window_start,window_end)
    
    segment = sig[window_start:window_end]
    samples = np.array(segment.get_array_of_samples())
    new_sig = samples.astype(np.float32)

    with WavProcessor() as proc:
        predictions = proc.get_predictions(16000, new_sig)

        for p in predictions:
            codes.append(p)
            pred_values = [x + y for x, y in zip(pred_values, get_labels(p[0]))]
            print(str(window_start / (60*1000)), ":", str(window_end / (60*1000)), " - ", df.loc[p[0]].values[1], p[1])
    
    current_length = window_start
    window_start += (window_size - overlap)
    window_end = (window_start + window_size)

(0, 6000)
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
('0', ':', '0', ' - ', 'Chime', 0.36180365085601807)
(3000, 9000)
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
(6000, 12000)
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
(9000, 15000)
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
('0', ':', '0', ' - ', 'Vehicle', 0.4404813349246979)
('0', ':', '0', ' - ', 'Car passing by', 0.2203224152326584)
('0', ':', '0', ' - ', 'Speech', 0.1655448079109192)
(12000, 18000)
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
('0', ':', '0', ' - ', 'Chime', 0.8761

INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
(123000, 129000)
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
(126000, 132000)
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
(129000, 135000)
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
('2', ':', '2', ' - ', 'Moo', 0.22909890115261078)
('2', ':', '2', ' - ', 'Cattle, bovinae', 0.18654510378837585)
('2', ':', '2', ' - ', 'Cricket', 0.1345018744468689)
('2', ':', '2', ' - ', 'Insect', 0.1025020182132721)
(132000, 138000)
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
(135000, 141000)
INFO:tensorflow:Res

INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
('3', ':', '4', ' - ', 'Vehicle', 0.20853456854820251)
(240000, 246000)
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
(243000, 249000)
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
('4', ':', '4', ' - ', 'Goat', 0.3902353048324585)
('4', ':', '4', ' - ', 'Animal', 0.2951367199420929)
('4', ':', '4', ' - ', 'Livestock, farm animals, working animals', 0.11170820891857147)
(246000, 252000)
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
(249000, 255000)
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
(252000, 258000)
INFO:tensorflow:Restoring parameters from models/vggish_mode

(357000, 363000)
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
('5', ':', '6', ' - ', 'Music', 0.9161608815193176)
('5', ':', '6', ' - ', 'Vehicle', 0.5143578052520752)
('5', ':', '6', ' - ', 'Truck', 0.324130654335022)
(360000, 366000)
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
(363000, 369000)
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
('6', ':', '6', ' - ', 'Car', 0.6028886437416077)
('6', ':', '6', ' - ', 'Vehicle', 0.23013024032115936)
('6', ':', '6', ' - ', 'Outside, urban or manmade', 0.20609411597251892)
('6', ':', '6', ' - ', 'Speech', 0.18349589407444)
('6', ':', '6', ' - ', 'Vehicle horn, car horn, honking', 0.14883825182914734)
(366000, 372000)
INFO:tensorflow:Restoring parameters from models/vggish_model.

INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
(477000, 483000)
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
('7', ':', '8', ' - ', 'Frog', 0.3732002079486847)
(480000, 486000)
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
('8', ':', '8', ' - ', 'Speech', 0.5325639247894287)
(483000, 489000)
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
('8', ':', '8', ' - ', 'Insect', 0.18725502490997314)
(486000, 492000)
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
(489000, 495000)
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow

INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
('9', ':', '9', ' - ', 'Wind chime', 0.1037454754114151)
(594000, 600000)
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
('9', ':', '10', ' - ', 'Frog', 0.11924735456705093)
(597000, 603000)
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
(600000, 606000)
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
(603000, 609000)
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
('10', ':', '10', ' - ', 'Outside, rural or natural', 0.1379925161600113)
(606000, 612000)
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube

INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
('11', ':', '11', ' - ', 'Frog', 0.12408356368541718)
(705000, 711000)
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
(708000, 714000)
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
(711000, 717000)
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
('11', ':', '11', ' - ', 'Wheeze', 0.28625574707984924)
(714000, 720000)
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
(717000, 723000)
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.

INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
(813000, 819000)
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
('13', ':', '13', ' - ', 'Insect', 0.14726638793945312)
(816000, 822000)
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
(819000, 825000)
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
(822000, 828000)
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
('13', ':', '13', ' - ', 'Speech', 0.9157306551933289)
('13', ':', '13', ' - ', 'Camera', 0.24959705770015717)
(825000, 831000)
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt
('13'

In [8]:
import math
df = pd.read_csv('labels.csv')
df = df.set_index('index')

val_list = {}

total_items = 0

for c in codes:
    total_items += 1
    
    if c[0] not in val_list:
        val_list[c[0]] = []
        
    val_list[c[0]].append(c[1])
    
items_dict = {}

for key, val in val_list.items():
    class_prob = len(val)/float(total_items)
    class_prob = 1 / (class_prob)
    confidence_prob = 1 / np.mean(val)
    harmonic = 2 / ( class_prob + confidence_prob )
    items_dict[key] = harmonic

_sorted = sorted(items_dict.items(), key=lambda x: x[1])

ultimate_list = {}

labels = list(df)[2:]

df_without_nan = df.dropna(thresh=3)
total_labels = df_without_nan.shape[0]


for label in labels:
    sound_probs = []
    for key, val in _sorted:
        if not (math.isnan(df.loc[key][label])):
            sound_probs.append(val)
        else:
            sound_probs.append(0.0)
    harmonic = 2 / (1 / (df_without_nan[label].count()/total_labels) + 1 / np.mean(sound_probs))
    ultimate_list[label] = harmonic


ultimate_sorted = sorted(ultimate_list.items(), key=lambda x: x[1])

for key, val in ultimate_sorted:
    print(key,val)
        
        

    
    
    
    


('Classroom', 0.0)
('Bathroom', 0.00025899291755294583)
('Public place', 0.0002616895832386206)
('Kitchen', 0.00052027810843233537)
('Work Shop', 0.00099114755651461169)
('Concert', 0.0027067148205781404)
('On water', 0.0029026004565930259)
('Work place(kantoor)', 0.0029631666358490493)
('Trainstation', 0.0031594215533992244)
('Home', 0.0032080871998430818)
('Roadside', 0.0056931761960995137)
('Nature', 0.026753549077926989)
('Park', 0.030531694435482905)
('Farm', 0.036478969747829354)


/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:43: RuntimeWarning: divide by zero encountered in long_scalars
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:43: RuntimeWarning: divide by zero encountered in double_scalars


In [ ]:
df_without_nan

In [ ]:
val_list = {}

total_items = 0

for c in codes:
    total_items += 1
    
    if c[0] not in val_list:
        val_list[c[0]] = []
        
    val_list[c[0]].append(c[1])

items_dict = {}

for key, val in val_list.items():
    mean = (len(val)/total_items) * np.mean(val)
    items_dict[key] = mean

_sorted = sorted(items_dict.items(), key=lambda x: x[1])

for key, val in _sorted:
    print(df.loc[key]['display_name'], val)

In [ ]:
pred_values

In [ ]:
df.loc[137]

In [ ]:
arr_index = np.argwhere(pred_values == np.amax(pred_values))
arr_index = arr_index.flatten().tolist()

labels = ['Living room', 'Work place(kantoor)', 'Classroom', 'Work Shop', 'Kitchen', 'Bathroom', 'Bedroom', 'Trainstation', 'Airfield', 'On water', 'Public place', 'Roadside', 'Concert', 'Farm', 'Nature', 'Park']

for x in arr_index:
    print(labels[int(x)])

print(pred_values)